# Preprocess
1. Data folders preprocess
2. Preprocess for segmentation
3. Preprocess for classification
--------

## 1. Data Folders preprocess
由于每个人的文件夹中，包括多个序列，其中只有一个序列为本研究使用的序列，且该文件夹中带有分割的（.nii.gz）文件。

    PAHSOS
    |-- patient1_name
        |--subfold1
            |--seq1
                |--xxxx.dcm
                |--xxxx.dcm
                |--patient1_name-SEG.nii.gz
            |--seq2
                |--xxxx.dcm
                |--xxxx.dcm
            |--seq3
                |--xxxx.dcm
                |--xxxx.dcm
    BCs
    HBV

为了后续使用方便，首先对原始数据进行处理，只保留一个序列的数据（.dcm）和对应的分割文件(.nii.gz)。将原始数据（Raw/）重新保存（simple/）。处理后的文件格式如下所示：

    PAHSOS
    |-- patient1_name
        |--xxxx.dcm
        |--xxxx.dcm
        |--patient1_name-SEG.nii.gz
    BCs
    HBV

* 由于这里有两批数据，因此分别进行处理，首先存到/simple_1/中，然后合并，并删除排除的病例，得到最终的simple_data/


In [2]:
import os
import glob
import SimpleITK as sitk
import numpy as np
import shutil

## 2. Preprocess for auto-segmentation



preprocess data. 
save(rename) following nnunet instructor

跑了两次，第一次是所有数据，在/mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/nnUNet_raw_data_base/nnUNet_raw_data/Task_LITS_ori。 

后一次把更改了两个数据重新跑了，在/mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/nnUNet_raw_data_base/nnUNet_raw_data/Task029_LITS。

然后把第二次的结果(idx=1,5)的output（output2）重新命名，替换到output中。

注意这里因为是从001开始命名的。因此对应的是002和006

好的，又有一个数据有问题 。 再重新跑一次XUANYUAN这个 030

In [2]:
import numpy as np
from scipy.ndimage import zoom
import numpy as np
import SimpleITK as sitk
import glob
import os

In [96]:
# function
def read_dcm(path):
    reader = sitk.ImageSeriesReader()
    dcm_series = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(dcm_series)
    img = reader.Execute()
    npy = sitk.GetArrayFromImage(img)
    return img, npy

def resample_to_spacing(npy_image, source_spacing, target_spacing, order=1):
    scale = np.array(source_spacing) / np.array(target_spacing)
    zoom_factor = np.array(target_spacing) / np.array(source_spacing)
    target_npy_image = zoom(npy_image, scale, order=order)
    return target_npy_image, zoom_factor, scale

def change_axes_of_image(npy_image, orientation, reorientation=[1, 1, 1]):
    if orientation[0] < 0:
        npy_image = np.flip(npy_image, axis=0)
        print('change_axes_of_image', orientation)
    if orientation[1] < 0:
        npy_image = np.flip(npy_image, axis=1)
        print('change_axes_of_image', orientation)
    if orientation[2] < 0:
        npy_image = np.flip(npy_image, axis=2)
        print('change_axes_of_image', orientation)
    return npy_image


In [97]:
# 用于seg数据的整理
'''
_image_ori : 原图
_image_aft_resample : 重采样后图
_label_ori : 原始label
_label_aft_resample : 重采样后的label。
'''

ori_img_path = '/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/'
# nnunet_img_save_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/nnUNet_raw_data_base/nnUNet_raw_data/Task029_LITS/imagesTs'
nnunet_img_save_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/nnUNet_raw_data_base/nnUNet_raw_data/Task029_LITS/imagesTs'
n = ['GANDUAN2']#, '111FANDE', 'ganduan']
cnt = 1
for subfolder in n:
    disease_cls_path = os.path.join(ori_img_path, f'{subfolder}/') 
    patients_path = glob.glob(os.path.join(disease_cls_path, '*'))

    for i, patient_path in enumerate(patients_path):

        patient_name = patient_path[patient_path.rfind('/')+1:]
        dcm_path = glob.glob(os.path.join(patient_path, '*/'))[0]
        print(patient_name)

        # if patient_name not in ('XUANYUAN'):
        #     continue

        
    #     # load img
        img, npy_img = read_dcm(dcm_path)

        # load info
        spacing = list(reversed(img.GetSpacing()))
        direction = img.GetDirection()
        direction = [direction[8], direction[4], direction[0]]

        print(f'loading {i}/{len(patients_path)} in folder {subfolder}: {patient_name}  spacing : {spacing} , direction : {direction}')
        
        # preprocess img : resample, reorientation
        aft_image, _, _ = resample_to_spacing(npy_img, spacing, [1, 1, 1])
        aft_image = change_axes_of_image(aft_image, direction, [1, 1, 1])
        nii_block = sitk.GetImageFromArray(aft_image)
       

        # save to required format
        if cnt<10:
            file_name = os.path.join(nnunet_img_save_path, f'test_00{cnt}_0000.nii.gz')
        elif cnt<100:
            file_name = os.path.join(nnunet_img_save_path, f'test_0{cnt}_0000.nii.gz')
        else:
            file_name = os.path.join(nnunet_img_save_path, f'test_{cnt}_0000.nii.gz')

        cnt += 1
        out = sitk.WriteImage(nii_block, file_name)
        print(f'saving files for {patient_name} as {file_name}')



DINGKOU
DINGXIZHENG
FENGZHAOYIN
GUORUANCHAO
LIGUOQIANG
LIJUN
LILINGSHAN
LIUXINMIN
LIUXIUYU
MALAN
MAMEILING
MENGXIANLIAN
MIAOSHUQIN
NIUSHUAN
NIUSHUAN2
SHIHAIXIA
SUNYONGXIN
SUNZHIQIANG
TIANQUANWU
WANGHAIXIN
XUANYUAN
loading 20/27 in folder GANDUAN2: XUANYUAN  spacing : [5.000006081891316, 0.984251976, 0.984251976] , direction : [1.0, 1.0, 1.0]


ImageSeriesReader (0x5586a032c3f0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0467142



saving files for XUANYUAN as /mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/nnUNet_raw_data_base/nnUNet_raw_data/Task029_LITS/imagesTs/test_001_0000.nii.gz
YANFENGCHAN
YANGUOQIANG
ZHANGCHANGHAI
ZHANGGUANGCHAO
ZHANGSHENGCAI
ZHUYOULIANG


## 3. Preprocess for classification

#### Step 1. reorientate & resample

- ori_data : /mnt/ExtData/pahsos/Data/simple_data

- 将处理后的图像和label保存成npy格式， /mnt/ExtData/pahsos/Data/preprocessed

In [105]:
import numpy as np
from scipy.ndimage import zoom
import numpy as np
import SimpleITK as sitk
import glob
import os
import nibabel as nib

In [161]:
# function
# def change_label(label):
#     '''seperate label 10 into 10, 11, 12'''

#     temp_label_10 = np.argwhere(label==10)
#     # print(len(temp_label_10))

#     x_range = np.unique(np.argwhere(label==10)[:, 0])
#     y_range = np.unique(np.argwhere(label==10)[:, 1])
#     z_range = np.unique(np.argwhere(label==10)[:, 2])
#     flag = 0
#     for range_unique in x_range, y_range, z_range:
#         if range_unique.shape[0] == 9:
#             # check
#             if range_unique[0] == range_unique[1]-1 == range_unique[2]-2 and range_unique[3] == range_unique[4]-1 == range_unique[5]-2 and range_unique[6] == range_unique[7]-1 == range_unique[8]-2:
#                 if range_unique is x_range:
#                     temp_label_10 = temp_label_10[temp_label_10[:,0].argsort()]
#                 elif range_unique is y_range:
#                     temp_label_10 = temp_label_10[temp_label_10[:,1].argsort()]
#                 else:
#                     temp_label_10 = temp_label_10[temp_label_10[:,2].argsort()]

#                 for m in range(0, 27):
                    
#                     label[temp_label_10[27+m][0], temp_label_10[27+m][1], temp_label_10[27+m][2]] = 11
#                     label[temp_label_10[54+m][0], temp_label_10[54+m][1], temp_label_10[54+m][2]] = 12
#                 flag = 1
#                 break
#     if flag == 0:
#         print('NO!')
#     return label  


def change_label_spe(label, patient_name):
    '''seperate label 10 into 10, 11, 12'''

    temp_label_10 = np.argwhere(label==10)
    # print(len(temp_label_10))

    x_range = np.unique(np.argwhere(label==10)[:, 0])
    y_range = np.unique(np.argwhere(label==10)[:, 1])
    z_range = np.unique(np.argwhere(label==10)[:, 2])
    flag = 0
    if (patient_name in ('LILINGSHAN', 'MENGXIANLIAN', 'SUNZHIQIANG', 'WANGHAIXIN')):
        if patient_name == 'LILINGSHAN':
            center = [[44, 222, 245], [44, 227, 220], [44, 262, 198]]
        elif patient_name == 'MENGXIANLIAN':
            center = [[87, 261, 238], [87, 255, 219], [86, 272, 191]]
        elif patient_name == 'SUNZHIQIANG':
            center = [[39, 250, 245], [38, 282, 211], [38, 263, 217]]
        elif patient_name == 'WANGHAIXIN':
            center = [[92, 264, 197], [92, 242, 210], [92, 238, 230]]
        else:
            print('WRONG LABEL!!')

        label[label == 10] = 0

        c = center[0]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 10
        c = center[1]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 11
        c = center[2]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 12
        flag = 1
    else:
        for range_unique in x_range, y_range, z_range:
            if range_unique.shape[0] == 9:
                # check
                if range_unique[0] == range_unique[1]-1 == range_unique[2]-2 and range_unique[3] == range_unique[4]-1 == range_unique[5]-2 and range_unique[6] == range_unique[7]-1 == range_unique[8]-2:
                    if range_unique is x_range:
                        temp_label_10 = temp_label_10[temp_label_10[:,0].argsort()]
                    elif range_unique is y_range:
                        temp_label_10 = temp_label_10[temp_label_10[:,1].argsort()]
                    else:
                        temp_label_10 = temp_label_10[temp_label_10[:,2].argsort()]

                    for m in range(0, 27):
                        
                        label[temp_label_10[27+m][0], temp_label_10[27+m][1], temp_label_10[27+m][2]] = 11
                        label[temp_label_10[54+m][0], temp_label_10[54+m][1], temp_label_10[54+m][2]] = 12
                    flag = 1
                    break
    if flag == 0:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!! WRONG LABEL !!!!!!!!!!!!!!!!!!!!!!!!!! ')
    return label  

def read_dcm(path):
    reader = sitk.ImageSeriesReader()
    dcm_series = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(dcm_series)
    img = reader.Execute()
    npy = sitk.GetArrayFromImage(img)
    return img, npy


def resample_to_spacing(npy_image, source_spacing, target_spacing, order=1):
    scale = np.array(source_spacing) / np.array(target_spacing)
    zoom_factor = np.array(target_spacing) / np.array(source_spacing)
    target_npy_image = zoom(npy_image, scale, order=order)
    return target_npy_image, zoom_factor, scale

def change_axes_of_image(npy_image, orientation, reorientation=[1, 1, 1]):
    if orientation[0] < 0:
        npy_image = np.flip(npy_image, axis=0)
    if orientation[1] < 0:
        npy_image = np.flip(npy_image, axis=1)
    if orientation[2] < 0:
        npy_image = np.flip(npy_image, axis=2)
    return npy_image

def resample_mask_to_spacing(npy_mask, source_spacing, target_spacing, num_label, order=1):
    # print('resampling mask')
    scale = np.array(source_spacing) / np.array(target_spacing)
    zoom_factor = np.array(target_spacing) / np.array(source_spacing)
    target_npy_mask = np.zeros_like(npy_mask)
    target_npy_mask = zoom(target_npy_mask, scale, order=order)
    # print(np.unique(npy_mask))
    for i in range(1, num_label + 1):
        # print(i)
        current_mask = npy_mask.copy()

        current_mask[current_mask != i] = 0
        current_mask[current_mask == i] = 1
        # print(len(np.argwhere(current_mask!=0)))

        current_mask = zoom(current_mask, scale, order=order)
        current_mask = (current_mask > 0.5).astype(np.uint8)
        # print(len(np.argwhere(current_mask!=0)))
        target_npy_mask[current_mask != 0] = i
    return target_npy_mask, zoom_factor, scale

#########################################################################################################################################################################################

这里发现有四个人len(np.argwhere(label == 10)) != 81 ,先对这几个人做一下修正。
这里先看一下这四个人的，然后把这些调整添加到change_label函数中。


In [147]:
看了itksnap，基本是有些误触，以下是修正后的三个点的中心
# a = sitk.GetArrayFromImage(sitk.ReadImage('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/GANDUAN2/LILINGSHAN/LILINGSHAN.nii.gz'))
#[ 44, 222, 245], [44, 227, 220], [44, 262, 198]
a = sitk.GetArrayFromImage(sitk.ReadImage('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/GANDUAN2/MENGXIANLIAN/MENGXIANLIAN.nii.gz'))
# [87, 261, 238], [87, 255, 219], [86, 272, 191]
# a = sitk.GetArrayFromImage(sitk.ReadImage('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/GANDUAN2/SUNZHIQIANG/SUNZHIQIANG.nii'))
# [39, 250, 245]  [38, 282, 211], [38, 263, 217]
# a = sitk.GetArrayFromImage(sitk.ReadImage('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/GANDUAN2/WANGHAIXIN/WANGHAIXIN.nii'))
# [92, 264, 197], [92, 242, 210], [92, 238, 230]

# np.argwhere(a == 10)


array([[ 85, 271, 190],
       [ 85, 271, 191],
       [ 85, 271, 192],
       [ 85, 272, 190],
       [ 85, 272, 191],
       [ 85, 272, 192],
       [ 85, 273, 190],
       [ 85, 273, 191],
       [ 85, 273, 192],
       [ 86, 254, 218],
       [ 86, 254, 219],
       [ 86, 254, 220],
       [ 86, 255, 218],
       [ 86, 255, 219],
       [ 86, 255, 220],
       [ 86, 256, 218],
       [ 86, 256, 219],
       [ 86, 256, 220],
       [ 86, 260, 237],
       [ 86, 260, 238],
       [ 86, 260, 239],
       [ 86, 261, 237],
       [ 86, 261, 238],
       [ 86, 261, 239],
       [ 86, 262, 237],
       [ 86, 262, 238],
       [ 86, 262, 239],
       [ 86, 271, 190],
       [ 86, 271, 191],
       [ 86, 271, 192],
       [ 86, 272, 190],
       [ 86, 272, 191],
       [ 86, 272, 192],
       [ 86, 273, 190],
       [ 86, 273, 191],
       [ 86, 273, 192],
       [ 87, 254, 218],
       [ 87, 254, 219],
       [ 87, 254, 220],
       [ 87, 255, 218],
       [ 87, 255, 219],
       [ 87, 255

In [159]:

def change_label_spe(label, patient_name):
    '''seperate label 10 into 10, 11, 12'''

    temp_label_10 = np.argwhere(label==10)
    # print(len(temp_label_10))

    x_range = np.unique(np.argwhere(label==10)[:, 0])
    y_range = np.unique(np.argwhere(label==10)[:, 1])
    z_range = np.unique(np.argwhere(label==10)[:, 2])
    flag = 0
    if (patient_name in ('LILINGSHAN', 'MENGXIANLIAN', 'SUNZHIQIANG', 'WANGHAIXIN')):
        if patient_name == 'LILINGSHAN':
            center = [[44, 222, 245], [44, 227, 220], [44, 262, 198]]
        elif patient_name == 'MENGXIANLIAN':
            center = [[87, 261, 238], [87, 255, 219], [86, 272, 191]]
        elif patient_name == 'SUNZHIQIANG':
            center = [[39, 250, 245], [38, 282, 211], [38, 263, 217]]
        elif patient_name == 'WANGHAIXIN':
            center = [[92, 264, 197], [92, 242, 210], [92, 238, 230]]
        else:
            print('WRONG LABEL!!')

        label[label == 10] = 0

        c = center[0]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 10
        c = center[1]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 11
        c = center[2]
        label[c[0] - 1: c[0] + 2, c[1] - 1: c[1] + 2, c[2] - 1: c[2] + 2] = 12
        flag = 1
    else:
        for range_unique in x_range, y_range, z_range:
            if range_unique.shape[0] == 9:
                # check
                if range_unique[0] == range_unique[1]-1 == range_unique[2]-2 and range_unique[3] == range_unique[4]-1 == range_unique[5]-2 and range_unique[6] == range_unique[7]-1 == range_unique[8]-2:
                    if range_unique is x_range:
                        temp_label_10 = temp_label_10[temp_label_10[:,0].argsort()]
                    elif range_unique is y_range:
                        temp_label_10 = temp_label_10[temp_label_10[:,1].argsort()]
                    else:
                        temp_label_10 = temp_label_10[temp_label_10[:,2].argsort()]

                    for m in range(0, 27):
                        
                        label[temp_label_10[27+m][0], temp_label_10[27+m][1], temp_label_10[27+m][2]] = 11
                        label[temp_label_10[54+m][0], temp_label_10[54+m][1], temp_label_10[54+m][2]] = 12
                    flag = 1
                    break
    if flag == 0:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!! WRONG LABEL !!!!!!!!!!!!!!!!!!!!!!!!!! ')
    return label  


ori_img_path = '/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/'
save_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/processed'
n = ['GANDUAN2', '111FANDE', 'ganduan']
cnt = 1
for subfolder in n:
    disease_cls_path = os.path.join(ori_img_path, f'{subfolder}/') 
    patients_path = glob.glob(os.path.join(disease_cls_path, '*'))

    for i, patient_path in enumerate(patients_path):

        patient_name = patient_path[patient_path.rfind('/')+1:]
        dcm_path = glob.glob(os.path.join(patient_path, '*/'))[0]
        # print('path', patient_name, patient_path)

        patient_save_path = os.path.join(save_path, patient_name)
        if not os.path.exists(patient_save_path):
            os.makedirs(patient_save_path)

        # load img
        img, npy_img = read_dcm(dcm_path)
        # load label
        label_path = glob.glob(os.path.join(patient_path, '*.nii*'))[0]
        if len(label_path) == 0:
            print(f'cant find label for patient {patient_name}')
        # npy_label = sitk.GetArrayFromImage(sitk.ReadImage(label_path))
        ## 因为郑州数据的label有的是nii有的是nii.gz 所以这里用nilabel读
        npy_label = nib.load(label_path).get_fdata()
        npy_label = npy_label.transpose(2, 1, 0)


        if npy_img.shape == npy_label.shape:
            ## for additional 
            if not (patient_name in ('LILINGSHAN', 'MENGXIANLIAN', 'SUNZHIQIANG', 'WANGHAIXIN')):
                continue
            # #load img info
            spacing = list(reversed(img.GetSpacing()))
            direction = img.GetDirection()
            direction = [direction[8], direction[4], direction[0]]

            # change label 10 into 10, 11, 12
            if len(np.argwhere(npy_label == 10)) != 81:
                print(patient_name, len(np.argwhere(npy_label == 10)), np.unique(npy_label))
            npy_label = change_label_spe(npy_label, patient_name)

            ## check效果！！
            print(len(np.argwhere(npy_label == 10)), len(np.argwhere(npy_label == 11)), len(np.argwhere(npy_label == 12)))

LILINGSHAN 87 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
27 27 27
MENGXIANLIAN 90 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
27 27 27


ImageSeriesReader (0x55869adb3df0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000987952



SUNZHIQIANG 82 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
27 27 27
WANGHAIXIN 90 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
27 27 27


ImageSeriesReader (0x55869adb3df0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0467142



###################################################################################################################

In [163]:
# 读取dcm文件 处理图像及勾画，并存储为npy

ori_img_path = '/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/'
save_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/processed'
n = ['GANDUAN2', '111FANDE', 'ganduan']
cnt = 1
for subfolder in n:
    disease_cls_path = os.path.join(ori_img_path, f'{subfolder}/') 
    patients_path = glob.glob(os.path.join(disease_cls_path, '*'))

    for i, patient_path in enumerate(patients_path):

        patient_name = patient_path[patient_path.rfind('/')+1:]
        dcm_path = glob.glob(os.path.join(patient_path, '*/'))[0]
        # print('path', patient_name, patient_path)

        patient_save_path = os.path.join(save_path, patient_name)
        if not os.path.exists(patient_save_path):
            os.makedirs(patient_save_path)

        # load img
        img, npy_img = read_dcm(dcm_path)
        # load label
        label_path = glob.glob(os.path.join(patient_path, '*.nii*'))[0]
        if len(label_path) == 0:
            print(f'cant find label for patient {patient_name}')
        # npy_label = sitk.GetArrayFromImage(sitk.ReadImage(label_path))
        ## 因为郑州数据的label有的是nii有的是nii.gz 所以这里用nilabel读
        npy_label = nib.load(label_path).get_fdata()
        npy_label = npy_label.transpose(2, 1, 0)


        if npy_img.shape == npy_label.shape:
            ## for additional 
            # if not (patient_name in ('LILINGSHAN', 'MENGXIANLIAN', 'SUNZHIQIANG')):
            #     continue
            # #load img info
            spacing = list(reversed(img.GetSpacing()))
            direction = img.GetDirection()
            direction = [direction[8], direction[4], direction[0]]

            # change label 10 into 10, 11, 12
            # if len(np.argwhere(npy_label == 10)) != 81:
            #     print(patient_name, len(np.argwhere(npy_label == 10)))
            # npy_label = change_label(npy_label)# 原始的，因为有四个有点问题，所以改成下面得了
            npy_label = change_label_spe(npy_label, patient_name)

            print(f'loading {i}/{len(patients_path)} : {patient_name}  spacing : {spacing} , direction : {direction}, {npy_img.shape}, {npy_label.shape}')
            
            # preprocess : resample, reorientation
            aft_image, _, _ = resample_to_spacing(npy_img, spacing, [1, 1, 1])
            aft_label, _, _ = resample_mask_to_spacing(npy_label, spacing, [1, 1, 1], 12)
            if direction != [1, 1, 1]:
                aft_image = change_axes_of_image(aft_image, direction, [1, 1, 1])
                aft_label = change_axes_of_image(aft_label, direction, [1, 1, 1])

            print(np.unique(npy_label), len(np.argwhere(npy_label!=0)), np.unique(aft_label), aft_label.shape)

            # saving
            np.save(os.path.join(patient_save_path, f'{patient_name}_image_ori.npy'), npy_img)
            np.save(os.path.join(patient_save_path, f'{patient_name}_image_aft_resample.npy'), aft_image)

            np.save(os.path.join(patient_save_path, f'{patient_name}_label_ori.npy'), npy_label)
            np.save(os.path.join(patient_save_path, f'{patient_name}_label_aft_resample.npy'), aft_label)
            print(f'🔵 {i} / {len(patients_path)} processing and saving npy files : {subfolder} - {patient_name}')
        else:
            print(f'🔴 {i} / {len(patients_path)} shape error : {subfolder} - {patient_name} ori img: {npy_img.shape}, ori label {npy_label.shape}')



loading 0/27 : DINGKOU  spacing : [5.0, 0.804688, 0.804688] , direction : [1.0, 1.0, 1.0], (130, 512, 512), (130, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 333 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (650, 412, 412)
🔵 0 / 27 processing and saving npy files : GANDUAN2 - DINGKOU
loading 1/27 : DINGXIZHENG  spacing : [5.0, 0.712890625, 0.712890625] , direction : [1.0, 1.0, 1.0], (78, 512, 512), (78, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (390, 365, 365)
🔵 1 / 27 processing and saving npy files : GANDUAN2 - DINGXIZHENG
loading 2/27 : FENGZHAOYIN  spacing : [5.0, 0.976562, 0.976562] , direction : [1.0, 1.0, 1.0], (77, 512, 512), (77, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (385, 500, 500)
🔵 2 / 27 processing and saving npy files : GANDUAN2 - FENGZHAOYIN
loading 3/27 : GUORUANCHAO  spacing : [5

ImageSeriesReader (0x55869cb7cef0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000987952



loading 15/27 : SHIHAIXIA  spacing : [4.999987951807229, 0.919922, 0.919922] , direction : [1.0, 1.0, 1.0], (84, 512, 512), (84, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (420, 471, 471)
🔵 15 / 27 processing and saving npy files : GANDUAN2 - SHIHAIXIA
loading 16/27 : SUNYONGXIN  spacing : [4.999998063492061, 0.78125, 0.78125] , direction : [1.0, 1.0, 1.0], (64, 512, 512), (64, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (320, 400, 400)
🔵 16 / 27 processing and saving npy files : GANDUAN2 - SUNYONGXIN
loading 17/27 : SUNZHIQIANG  spacing : [5.0, 0.782, 0.782] , direction : [1.0, 1.0, 1.0], (46, 512, 512), (46, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 328 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (230, 400, 400)
🔵 17 / 27 processing and saving npy files : GANDUAN2 - SUNZHIQIANG
loading 18/27 : TIAN

ImageSeriesReader (0x55869cb7cef0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0467142



loading 20/27 : XUANYUAN  spacing : [5.000006081891316, 0.984251976, 0.984251976] , direction : [1.0, 1.0, 1.0], (94, 508, 508), (94, 508, 508)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (470, 500, 500)
🔵 20 / 27 processing and saving npy files : GANDUAN2 - XUANYUAN
loading 21/27 : YANFENGCHAN  spacing : [5.0, 0.976562, 0.976562] , direction : [1.0, 1.0, 1.0], (96, 512, 512), (96, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (480, 500, 500)
🔵 21 / 27 processing and saving npy files : GANDUAN2 - YANFENGCHAN
loading 22/27 : YANGUOQIANG  spacing : [5.0, 0.890625, 0.890625] , direction : [1.0, 1.0, 1.0], (148, 512, 512), (148, 512, 512)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 324 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] (740, 456, 456)
🔵 22 / 27 processing and saving npy files : GANDUAN2 - YANGUOQIANG
loading 23/27 : ZH

这里zhouweimeng的mask和label大小不一样，后续已经在simple1以及simple_data中更正，并重新单独处理、生成npy文件

### !!!! nibabel读取的需要transpose(2, 1, 0 )才能和sitk读的一样！

In [13]:
# a = sitk.GetArrayFromImage(sitk.ReadImage('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/111FANDE/GUOJINLIANG/GUOJINLIANG.nii.gz'))
# import nibabel as nib
# b = nib.load('/media/yinn147/My Passport/2、血吡咯蛋白加合物联合影像组学诊断PAs-HSOS的诊断模型建立与评价（高老师医院课题）/郑州标注/111FANDE/GUOJINLIANG/GUOJINLIANG.nii.gz').get_fdata()

# print(a.shape, b.shape)
# len(np.argwhere(b.transpose(2, 1, 0) != a))


(57, 508, 508) (508, 508, 57)


0

### Step 2. cutting blocks

这里 '/mnt/ExtData/pahsos/mask_name.csv' 是之前用于seg处理时，将文件换成test_001_0000.nii.gz格式时的姓名和对应序号关系。这里按照同样的方式遍历，和csv是按顺序一一对应的

根据给定的block_size，以及是否使用segmentation作为mask，分别切取12个label对应的block，保存在/preprocess/[cls_type]/[patient_name]/block[block_size]_masked[masked_block0]/中。分别为 1.npy - 12.npy

In [164]:
# 做验证使用，确认两者一一对应，不需要跑
import pandas as pd

match_list = pd.read_csv('mask_name.csv', header=None)
match_list = np.array(match_list)
print(match_list.shape)

preprocessed_npy_path = '/mnt/ExtData/pahsos/Data/preprocessed'
masked = True # False


(48, 1)


In [165]:
# function
def resample_to_size(npy_image, target_size, order=1):
    source_size = npy_image.shape
    scale = np.array(target_size) / source_size
    zoom_factor = source_size / np.array(target_size)
    target_npy_image = zoom(npy_image, scale, order=order)
    return target_npy_image

def get_block(img, mask, blocksize=64):
    
    center = [int(np.mean((np.argwhere(mask != 0))[:, 0])), int(np.mean((np.argwhere(mask != 0))[:, 1])), int(np.mean((np.argwhere(mask != 0))[:, 2]))]
    # print(img.shape, center)

    block_size = blocksize
    block = np.zeros((block_size, block_size, block_size))
    
    if block_size >= img.shape[0] or block_size >= img.shape[1] or block_size >= img.shape[2]:
        print('too big!')
        trgt_size_x = max(block_size, img.shape[0])
        trgt_size_y = max(block_size, img.shape[1])
        trgt_size_z = max(block_size, img.shape[2])
        img = resample_to_size(img, (trgt_size_x, trgt_size_y, trgt_size_z))
        mask = resample_to_size(mask, (trgt_size_x, trgt_size_y, trgt_size_z))
    # else:
    ## 处理部分从center取block，会取到外面的情况
    if center[0] + int((block_size-1)/2) >= img.shape[0]:
        center[0] = img.shape[0] - int((block_size-1)/2) 
        # print('1')
    if center[1] + int((block_size-1)/2) >= img.shape[1]:
        center[1] = img.shape[1] - int((block_size-1)/2)
        # print('2')
    if center[2] + int((block_size-1)/2) >= img.shape[2]:
        center[2] = img.shape[2] - int((block_size-1)/2)
        # print('3')

    if center[0] - int((block_size-1)/2) <= 0:
        center[0] = int((block_size-1)/2)
        # print('4')
    if center[1] - int((block_size-1)/2) <= 0:
        center[1] = int((block_size-1)/2)
        # print('5')
    if center[2] - int((block_size-1)/2) <= 0:
        center[2] = int((block_size-1)/2)
        # print('6')
    
    block = img[center[0] - int((block_size-1)/2): center[0] + block_size - int((block_size-1)/2), 
                center[1] - int((block_size-1)/2): center[1] + block_size - int((block_size-1)/2),
                center[2] - int((block_size-1)/2): center[2] + block_size - int((block_size-1)/2)]

    return block


In [167]:
# CROP BLOCK
preprocessed_npy_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/processed'
#这里默认保存在原始preprocess的文件夹中
seg_output_path = '/mnt/ExtData/workspace/pahsos/new_zhengzhou/segmentation/output'

masked_block = True # False
block_sizes = [64, 96, 128, 160]

for block_size in block_sizes:
    seg_output_list = sorted(glob.glob(os.path.join(seg_output_path, '*nii.gz')))

    # patient_path = np.sort(glob.glob(os.path.join(preprocessed_npy_path, '*')))

    for patient_idx in range(match_list.shape[0]):     

        patient_path = os.path.join(preprocessed_npy_path, match_list[patient_idx][0])  
        # 创建block保存文件夹
        block_save_path = os.path.join(patient_path, f'block{block_size}_masked{masked_block}_new')
        if not os.path.exists(block_save_path):
            os.makedirs(block_save_path)
        
        patient_name = match_list[patient_idx][0]

        # if os.path.exists(os.path.join(block_save_path, f'{i}.npy')):
        #     print(f'already process {patient_name}')
        #     continue

        seg_nii_path = seg_output_list[patient_idx]
        img_npy_path = os.path.join(patient_path, f'{patient_name}_image_aft_resample.npy')
        mask_npy_path = os.path.join(patient_path, f'{patient_name}_label_aft_resample.npy')

        seg = sitk.GetArrayFromImage(sitk.ReadImage(seg_nii_path))

        seg[seg>=1] = 1
        img = np.load(img_npy_path)
        mask = np.load(mask_npy_path)

        if seg.shape != img.shape or img.shape != mask.shape or seg.shape != mask.shape:
            print(f'🔴 {patient_idx} / {len(patient_path)} shape error : {patient_name} seg : {seg.shape} img : {img.shape}, ori label : {mask.shape}')
        else:
            
            if masked_block:
                img = img * seg

            for i in range(1, len(np.unique(mask))):
                one_label = np.zeros(mask.shape)
                one_label[mask==i] = 1
                if len(np.argwhere(one_label != 0)) == 0:
                    print(f'🔴 {patient_idx} / {match_list.shape[0]} - {patient_name} mask {i} error')
                else:
                    block = get_block(img, one_label, block_size)
                    np.save(os.path.join(block_save_path, f'{i}.npy'), block)
            if i == len(np.unique(mask))-1:
                print(f'🔵 {patient_idx} / {match_list.shape[0]} - {patient_name}')
            else:
                print(f'please check mask {np.unique(mask)}')
 



🔵 0 / 48 - GUOJINLIANG
🔵 1 / 48 - LUFENGDE
🔵 2 / 48 - NIUSHUAN3
🔵 3 / 48 - WANGXIAOHUA
🔵 4 / 48 - XIAOJINZHONG
🔵 5 / 48 - XIEDONGMEI
🔵 6 / 48 - XIETAO
🔵 7 / 48 - XUPENGPENG
🔵 8 / 48 - ZHANGXIUHUA
🔵 9 / 48 - DINGKOU
🔵 10 / 48 - DINGXIZHENG
🔵 11 / 48 - FENGZHAOYIN
🔵 12 / 48 - GUORUANCHAO
🔵 13 / 48 - LIGUOQIANG
🔵 14 / 48 - LIJUN
🔵 15 / 48 - LILINGSHAN
🔵 16 / 48 - LIUXINMIN
🔵 17 / 48 - LIUXIUYU
🔵 18 / 48 - MALAN
🔵 19 / 48 - MAMEILING
🔵 20 / 48 - MENGXIANLIAN
🔵 21 / 48 - MIAOSHUQIN
🔵 22 / 48 - NIUSHUAN
🔵 23 / 48 - NIUSHUAN2
🔵 24 / 48 - SHIHAIXIA
🔵 25 / 48 - SUNYONGXIN
🔵 26 / 48 - SUNZHIQIANG
🔵 27 / 48 - TIANQUANWU
🔵 28 / 48 - WANGHAIXIN
🔵 29 / 48 - XUANYUAN
🔵 30 / 48 - YANFENGCHAN
🔵 31 / 48 - YANGUOQIANG
🔵 32 / 48 - ZHANGCHANGHAI
🔵 33 / 48 - ZHANGGUANGCHAO
🔵 34 / 48 - ZHANGSHENGCAI
🔵 35 / 48 - ZHUYOULIANG
🔵 36 / 48 - CHENGUITANG
🔵 37 / 48 - FANGJIALI
🔵 38 / 48 - HANTIEZHU
🔵 39 / 48 - JINYUEE
🔵 40 / 48 - LIQINGXI
🔵 41 / 48 - OUXIANCHEN
🔵 42 / 48 - SHIHEWANG
🔵 43 / 48 - TONGNANGZI
🔵 44 / 48 -